# Polar Scalogram Projects

In [1]:
import wfdb
import matplotlib as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import resample
import numpy as np
import neurokit2 as nk
import pywt
import seaborn as sns
import os


In [2]:
# 현재 경로 불러오기 = os.getcwd()
path = os.getcwd()
os.chdir(path+ "\\training2017")
print(os.getcwd())


d:\Code Works\Work\vscode\ecgproject\Spectrogram\training2017


In [3]:
csv='REFERENCE.csv'
data = pd.read_csv(csv, header=None, names=['ID', 'Label'])

> ECG READ

In [4]:
def ecg_read(patient_id):
    record = wfdb.rdrecord(patient_id) 
    ecg_signal = record.p_signal[:,0]  
    fs = record.fs
    return ecg_signal, fs

> Labeling:  Dictionary for ALL data

In [5]:
A =[] # Atrial Fibrillation
N= [] # Normal Sinus rhythm
O = [] # Other rhythm
I = [] # Noisy

for id, lb in zip (data['ID'], data['Label'] ): #v2 data
         
         if lb =='A':
            A.append(id)
            
         elif lb =='N':
            N.append(id)
         
         elif lb =='O':
            O.append(id)
        
         elif lb =='~':
            I.append(id)   

In [6]:
dic_all = {'Atrial Fibrillation': A, 'Normal Sinus Rhythm': N, 'Other Rhythm': O, 'Noisy': I}
# print(len(A))
# print(len(N))
# print(len(O))
# print(len(I))
# print(len(A)+len(N)+len(O)+len(I))

> Dictionary for 30s data

In [7]:
# A_30 =[] # Atrial Fibrillation
# N_30= [] # Normal Sinus rhythm
# O_30 = [] # Other rhythm
# I_30 = [] # Noisy
# id_30=[]


# # 30초 아이디 찾기기
# for id, lb in zip(data['ID'], data['Label']):

#     ecg_signal, fs=ecg_read(id)
#     time = np.arange(ecg_signal.shape[0]) / fs

#     if int(len(time)/fs) == 30:
#         id_30.append(id)


# # 30초 아이디 라벨 매칭칭
# for id, lb in zip(data['ID'], data['Label']):  # 원본 데이터에서 ID와 라벨을 반복
#     if id in id_30:  # ID가 id_30 리스트에 있는지 확인
#         if lb == 'A':
#             A_30.append(id)
#         elif lb == 'N':
#             N_30.append(id)
#         elif lb == 'O':
#             O_30.append(id)
#         elif lb == '~':
#             I_30.append(id)


# # 30초 데이터에 관한 dictionary
# dic_30 = {
#     "Atrial Fibrillation": A_30,
#     "Normal Sinus Rhythm": N_30,
#     "Other Rhythm": O_30,
#     "Noisy": I_30
# }

> nomalization

In [8]:

def normalize_ecg(ecg_signal):
    min_val = np.min(ecg_signal)
    max_val = np.max(ecg_signal)
    normalized_signal = (ecg_signal - min_val) / (max_val - min_val)
    return normalized_signal


> Resampling


In [9]:
# Resample ECG signal from original sampling rate to target sampling rate -> 해상도
def resample_ecg_signal(ecg_signal, original_fs, target_fs):
    num_samples = int(len(ecg_signal) * target_fs / original_fs)
    resampled_signal = resample(ecg_signal, num_samples)
    return resampled_signal

> PT
#### Acknowledgements ####

This project utilizes code from [Pramod07Ch](https://github.com/Pramod07Ch). Their contributions have been invaluable to the development of this project. You can find the original repository [here](https://github.com/Pramod07Ch/Pan-Tompkins-algorithm-python).

In [10]:
from scipy.signal import butter, filtfilt
class Pan_tompkins:
    """ Implementationof Pan Tompkins Algorithm.

    Noise cancellation (bandpass filter) -> Derivative step -> Squaring and integration.

    Params:
        data (array) : ECG data
        sampling rate (int)
    returns:
        Integrated signal (array) : This signal can be used to detect peaks


    ----------------------------------------
    HOW TO USE ?
    Eg.

    ECG_data = [4, 7, 80, 78, 9], sampling  =2000
    
    call : 
       signal = Pan_tompkins(ECG_data, sampling).fit()

    ----------------------------------------
    
    """
    def __init__(self, data, sample_rate):

        self.data = data
        self.sample_rate = sample_rate


    def fit(self, normalized_cut_offs=None, butter_filter_order=2, padlen=150, window_size=None):
        ''' Fit the signal according to algorithm and returns integrated signal
        
        '''
        # 1.Noise cancellationusing bandpass filter
        self.filtered_BandPass = self.band_pass_filter(normalized_cut_offs, butter_filter_order, padlen)
        
        # 2.derivate filter to get slpor of the QRS
        self.derviate_pass = self.derivative_filter()

        # 3.Squaring to enhance dominant peaks in QRS
        self.square_pass = self.squaring()

        # 4.To get info about QRS complex
        self.integrated_signal = self.moving_window_integration( window_size)

        return self.integrated_signal


    def band_pass_filter(self, normalized_cut_offs=None, butter_filter_order=2, padlen=150):
        ''' Band pass filter for Pan tompkins algorithm
            with a bandpass setting of 5 to 20 Hz

            params:
                normalized_cut_offs (list) : bandpass setting canbe changed here
                bandpass filte rorder (int) : deffault 2
                padlen (int) : padding length for data , default = 150
                        scipy default value = 2 * max(len(a coeff, b coeff))

            return:
                filtered_BandPass (array)
        '''

        # Calculate nyquist sample rate and cutoffs
        nyquist_sample_rate = self.sample_rate / 2

        # calculate cutoffs
        if normalized_cut_offs is None:
            normalized_cut_offs = [5/nyquist_sample_rate, 15/nyquist_sample_rate]
        else:
            assert type(self.sample_rate ) is list, "Cutoffs should be a list with [low, high] values"

        # butter coefficinets 
        b_coeff, a_coeff = butter(butter_filter_order, normalized_cut_offs, btype='bandpass')[:2]

        # apply forward and backward filter
        filtered_BandPass = filtfilt(b_coeff, a_coeff, self.data, padlen=padlen)
        
        return filtered_BandPass


    def derivative_filter(self):
        ''' Derivative filter

        params:
            filtered_BandPass (array) : outputof bandpass filter
        return:
            derivative_pass (array)
        '''

        # apply differentiation
        derviate_pass= np.diff(self.band_pass_filter())

       

        return derviate_pass


    def squaring(self):
        ''' squaring application on derivate filter output data

        params:

        return:
            square_pass (array)
        '''

        # apply squaring
        square_pass= self.derivative_filter() **2

        return square_pass 


    def moving_window_integration(self, window_size=None):
        ''' Moving avergae filter 

        Params:
            window_size (int) : no. of samples to average, if not provided : 0.08 * sample rate
            sample_rate (int) : should be given if window_size is not given  
        return:
            integrated_signal (array)
        '''

        if window_size is None:
            assert self.sample_rate is not None, "if window size is None, sampling rate should be given"
            window_size = int(0.08 * int(self.sample_rate))  # given in paper 150ms as a window size
        

        # define integrated signal
        integrated_signal = np.zeros_like(self.squaring())

        # cumulative sum of signal
        cumulative_sum = self.squaring().cumsum()

        # estimationof area/ integral below the curve deifnes the data
        integrated_signal[window_size:] = (cumulative_sum[window_size:] - cumulative_sum[:-window_size]) / window_size

        integrated_signal[:window_size] = cumulative_sum[:window_size] / np.arange(1, window_size + 1)

        return integrated_signal

## save function

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import scipy.signal as signal


# Save polar spectrogram
def save_polar_spectrogram(polar_data, T, R, save_path, id, image_size, dpi, vmin=None, vmax=None, cmap='turbo'):
    fig_size_inch = (image_size[0] / dpi, image_size[1] / dpi)
    fig, ax = plt.subplots(figsize=fig_size_inch, dpi=dpi, subplot_kw={'projection': 'polar'})
    
    
    if vmin is not None and vmax is not None:
        c = ax.pcolormesh(T, R, polar_data, shading='auto', cmap=cmap, vmin=vmin, vmax=vmax)
    else:
        c = ax.pcolormesh(T, R, polar_data, shading='auto', cmap=cmap)
    
    # Remove axis lines, labels, and ticks
    ax.set_theta_zero_location('N')  # 0 degrees at the top
    ax.set_theta_direction(-1)  # Clockwise direction
    ax.axis('off')
    
    # Ensure there is no padding and no border around the image
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    ax.margins(0, 0)
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    
    # Save the figure as a PNG image with a transparent background
    plt.savefig(f'{save_path}/{id}.png', format='png', transparent=True, bbox_inches='tight', pad_inches=0)
    plt.savefig(os.path.join(save_path, id), transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()


> Save Path

In [12]:
# 필요에 따라 설정
base_save_path= path
print(base_save_path)

d:\Code Works\Work\vscode\ecgproject\Spectrogram


> Image parameter

In [13]:
# ECG 전처리 관련 파라미터
target_fs = 300  # ecg resampling(해상도), 기본 300 
Nomaliztion = None
PT_denoised = False# Whether to apply Pantompkins denoising
padding_length_s = None # 0.25씩 앞뒤로  패딩, 총 0.5 패딩
alpha = 0.05# Tukey window  # 0.05 30초에서는 이게 나을지도? #0.2 실험용 이미지 or 0

# Scalogram 관련 파라미터
max_scale = 128
wavelet = 'mexh'
dbscale= False

# 이미지 저장관련 파라미터
image_size = (224,224)
dpi = 100
vmin= None#0  
vmax=None# 0.87   #1.45   #1.33  db 최대값
cmap = 'turbo'   #'nipy_spectral'#'turbo'  #nipy_spectral


### img save (30s) : Polar

In [14]:
# for label, patient_ids in dic_30.items():
#     # 해당 리듬에 대한 폴더 생성
#     if label == "Atrial Fibrillation":
#         folder_name = "A"
#     elif label == "Normal Sinus Rhythm":
#         folder_name = "N"
#     elif label == "Other Rhythm":
#         folder_name = "O"
#     elif label == "Noisy":
#         folder_name = "~"

#     save_path_directory = os.path.join(base_save_path, folder_name)
#     if not os.path.exists(save_path_directory):
#         os.makedirs(save_path_directory)

#     # images_created=0    

#     for patient_id in patient_ids:
#         save_filename = f'{patient_id}'
#         save_path_img = os.path.join(save_path_directory, f'{save_filename}.png')

#         if os.path.exists(save_path_img):
#             print(f"Skipping {patient_id} already exist.")
#             continue

#              # 폴더당 최대 이미지 생성 시
#         # if images_created >= 10:
#         #     break
#         # images_created+=1

#         print(f"Processing {patient_id}")
        
#         ecg_signal, fs = ecg_read(patient_id)
        
#         if np.isnan(ecg_signal).any():
#             ecg_signal = np.nan_to_num(ecg_signal, nan=0.0)


#         if PT_denoised:
#             # try:
#             #     signals, info = nk.ecg_process(ecg_signal, sampling_rate=fs, method='pantompkins1985')
#             #     if np.isnan(signals["ECG_Clean"]).any():
#             #         signals["ECG_Clean"] = np.nan_to_num(signals["ECG_Clean"], nan=0.0)
#             #     ecg_signal = signals["ECG_Clean"]
#             # except Exception as e:
#             #     print(f"Error processing ECG signal with Pantompkins: {e}")
#             #     try:
#             #         ecg_filtered = nk.ecg_clean(ecg_signal, sampling_rate=fs, method="biosppy")
#             #         signals, info = nk.ecg_process(ecg_filtered, sampling_rate=fs, method='pantompkins1985')
#             #         if np.isnan(signals["ECG_Clean"]).any():
#             #             signals["ECG_Clean"] = np.nan_to_num(signals["ECG_Clean"], nan=0.0)
#             #         ecg_signal = signals["ECG_Clean"]
#             #     except Exception as e:
#             #         print(f"Error processing ECG signal with Pantompkins after biosppy filtering: {e}")

#             pt_tompkins = Pan_tompkins(ecg_signal,fs).fit()
#             ecg_signal = pt_tompkins

#         if Nomaliztion:
#             ecg_signal= normalize_ecg(ecg_signal)       
                  

#         if target_fs != 300:
#             ecg_signal = resample_ecg_signal(ecg_signal, fs, target_fs)
#             fs = target_fs 
        
#         if padding_length_s ==0 or padding_length_s == None :
#             padding_length =0
#             ecg_signal_padded=ecg_signal
#         else:    
#             padding_length = int(target_fs / padding_length_s)
#             # ecg_signal_padded = np.pad(ecg_signal, (padding_length, padding_length), 'constant', constant_values=(0, 0))
#             ecg_signal_padded = np.pad(ecg_signal, (padding_length, padding_length), mode='reflect')

#         if alpha:
#             tukey_window = signal.windows.tukey(len(ecg_signal_padded), alpha=alpha)
#             ecg_signal_padded *= tukey_window
#         else:    
#             ecg_signal_padded=ecg_signal_padded

#         scales = np.arange(1, max_scale + 1)
#         wavelet_coeffs, freqs = pywt.cwt(ecg_signal_padded, scales, wavelet, sampling_period=1/fs)

#         if np.isnan(wavelet_coeffs).any():
#             wavelet_coeffs = np.nan_to_num(wavelet_coeffs, nan=0.0)

#         times_padded = np.linspace(0, len(ecg_signal_padded) / fs, len(ecg_signal_padded))
#         theta = 2 * np.pi * times_padded / max(times_padded)
#         rho = np.linspace(0.1, max_scale, max_scale)
#         T, R = np.meshgrid(theta, rho)



#         # polar_data = np.flipud(np.abs(wavelet_coeffs))
#         # DB SACLE
#         energy = np.abs(wavelet_coeffs) ** 2
#         energy_db = 10* np.log10(energy + 1e-10)
#         polar_data = np.flipud(energy_db)


#         save_polar_spectrogram(polar_data, T, R, save_path_directory, save_filename, image_size=image_size, dpi=dpi,vmin=vmin, vmax=vmax, cmap=cmap)
#         # save_polar_spectrogram(polar_data, T, R, save_path, save_filename, image_size=image_size, dpi=dpi, vmin=vmin, vmax=vmax, cmap=cmap)

### img save (30s) : None-Polar(Square)

In [15]:
# def save_scalogram(coefficients, save_path, id, cmap, image_size, dpi):
#     fig_size_inch = (image_size[0] / dpi, image_size[1] / dpi)
#     fig, ax = plt.subplots(figsize=fig_size_inch, dpi=dpi)
    
# # Apply dB scale for the scalogram
# if dbsacle:
#     energy = np.abs(coefficients) ** 2
#     energy_db = 10 * np.log10(energy + 1e-10)
#     c = ax.imshow(energy_db, cmap=cmap, aspect='auto')

# # None db sacle
# else:

#     c = ax.imshow(np.abs(coefficients), cmap=cmap, aspect='auto')
    
#     # Remove axis for clean image
#     ax.axis('off')
    
#     # Adjust layout to avoid padding
#     plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
#     ax.margins(0, 0)
#     ax.xaxis.set_major_locator(plt.NullLocator())
#     ax.yaxis.set_major_locator(plt.NullLocator())
    
#     # Save the scalogram image
#     plt.savefig(f'{save_path}/{id}.png', format='png', bbox_inches='tight', pad_inches=0, transparent=True)
#     plt.close()


# for label, patient_ids in dic_30.items():
#     if label == "Atrial Fibrillation":
#         folder_name = "A"
#     elif label == "Normal Sinus Rhythm":
#         folder_name = "N"
#     elif label == "Other Rhythm":
#         folder_name = "O"
#     elif label == "Noisy":
#         folder_name = "~"

#     save_path_directory = os.path.join(base_save_path, folder_name)
#     if not os.path.exists(save_path_directory):
#         os.makedirs(save_path_directory)

#     for patient_id in patient_ids:
#         save_filename = f'{patient_id}'
#         save_path_img = os.path.join(save_path_directory, f'{save_filename}.png')

#         if os.path.exists(save_path_img):
#             print(f"Skipping {patient_id}, already exists.")
#             continue

#         print(f"Processing {patient_id}")
        
#         # Load and preprocess ECG data
#         ecg_signal, fs = ecg_read(patient_id)
        
#         if np.isnan(ecg_signal).any():
#             ecg_signal = np.nan_to_num(ecg_signal, nan=0.0)

#         if PT_denoised:
#             pt_tompkins = Pan_tompkins(ecg_signal, fs).fit()
#             ecg_signal = pt_tompkins

#         if Nomaliztion:
#             ecg_signal = normalize_ecg(ecg_signal)                 

#         if target_fs != 300:
#             ecg_signal = resample_ecg_signal(ecg_signal, fs, target_fs)
#             fs = target_fs 

#         scales = np.arange(1, max_scale + 1)
#         wavelet_coeffs, freqs = pywt.cwt(ecg_signal, scales, wavelet, sampling_period=1/fs)

#         if np.isnan(wavelet_coeffs).any():
#             wavelet_coeffs = np.nan_to_num(wavelet_coeffs, nan=0.0)

#         # Save the scalogram
#         save_scalogram(wavelet_coeffs, save_path_directory, save_filename, cmap=cmap, image_size=image_size, dpi=dpi, dbscale=True)

### img save (All) : Polar

In [ ]:
for label, patient_ids in dic_all.items():
    # 해당 리듬에 대한 폴더 생성
    if label == "Atrial Fibrillation":
        folder_name = "A"
    elif label == "Normal Sinus Rhythm":
        folder_name = "N"
    elif label == "Other Rhythm":
        folder_name = "O"
    elif label == "Noisy":
        folder_name = "~"

    save_path_directory = os.path.join(base_save_path, folder_name)
    if not os.path.exists(save_path_directory):
        os.makedirs(save_path_directory)

    # images_created=0    

    for patient_id in patient_ids:
        save_filename = f'{patient_id}'
        save_path_img = os.path.join(save_path_directory, f'{save_filename}.png')

        if os.path.exists(save_path_img):
            print(f"Skipping {patient_id} already exist.")
            continue

             # 폴더당 최대 이미지 생성 시
        # if images_created >= 10:
        #     break
        # images_created+=1

        print(f"Processing {patient_id}")
        
        ecg_signal, fs = ecg_read(patient_id)
        
        if np.isnan(ecg_signal).any():
            ecg_signal = np.nan_to_num(ecg_signal, nan=0.0)


        if PT_denoised:
            # try:
            #     signals, info = nk.ecg_process(ecg_signal, sampling_rate=fs, method='pantompkins1985')
            #     if np.isnan(signals["ECG_Clean"]).any():
            #         signals["ECG_Clean"] = np.nan_to_num(signals["ECG_Clean"], nan=0.0)
            #     ecg_signal = signals["ECG_Clean"]
            # except Exception as e:
            #     print(f"Error processing ECG signal with Pantompkins: {e}")
            #     try:
            #         ecg_filtered = nk.ecg_clean(ecg_signal, sampling_rate=fs, method="biosppy")
            #         signals, info = nk.ecg_process(ecg_filtered, sampling_rate=fs, method='pantompkins1985')
            #         if np.isnan(signals["ECG_Clean"]).any():
            #             signals["ECG_Clean"] = np.nan_to_num(signals["ECG_Clean"], nan=0.0)
            #         ecg_signal = signals["ECG_Clean"]
            #     except Exception as e:
            #         print(f"Error processing ECG signal with Pantompkins after biosppy filtering: {e}")

            pt_tompkins = Pan_tompkins(ecg_signal,fs).fit()
            ecg_signal = pt_tompkins

        if Nomaliztion:
            ecg_signal= normalize_ecg(ecg_signal)       
                  

        if target_fs != 300:
            ecg_signal = resample_ecg_signal(ecg_signal, fs, target_fs)
            fs = target_fs 
        
        if padding_length_s ==0 or padding_length_s == None :
            padding_length =0
            ecg_signal_padded=ecg_signal
        else:    
            padding_length = int(target_fs / padding_length_s)
            # ecg_signal_padded = np.pad(ecg_signal, (padding_length, padding_length), 'constant', constant_values=(0, 0))
            ecg_signal_padded = np.pad(ecg_signal, (padding_length, padding_length), mode='reflect')

        if alpha:
            tukey_window = signal.windows.tukey(len(ecg_signal_padded), alpha=alpha)
            ecg_signal_padded *= tukey_window
        else:    
            ecg_signal_padded=ecg_signal_padded

        scales = np.arange(1, max_scale + 1)
        wavelet_coeffs, freqs = pywt.cwt(ecg_signal_padded, scales, wavelet, sampling_period=1/fs)

        if np.isnan(wavelet_coeffs).any():
            wavelet_coeffs = np.nan_to_num(wavelet_coeffs, nan=0.0)

        times_padded = np.linspace(0, len(ecg_signal_padded) / fs, len(ecg_signal_padded))
        theta = 2 * np.pi * times_padded / max(times_padded)
        rho = np.linspace(0.1, max_scale, max_scale)
        T, R = np.meshgrid(theta, rho)



        polar_data = np.flipud(np.abs(wavelet_coeffs))
        # DB SACLE
        # energy = np.abs(wavelet_coeffs) ** 2
        # energy_db = 10* np.log10(energy + 1e-10)
        # polar_data = np.flipud(energy_db)


        save_polar_spectrogram(polar_data, T, R, save_path_directory, save_filename, image_size=image_size, dpi=dpi,vmin=vmin, vmax=vmax, cmap=cmap)
        # save_polar_spectrogram(polar_data, T, R, save_path, save_filename, image_size=image_size, dpi=dpi, vmin=vmin, vmax=vmax, cmap=cmap)

Processing A00004
Processing A00005
Processing A00009
Processing A00015
Processing A00027
Processing A00054
Processing A00067
Processing A00071
Processing A00087
Processing A00090
Processing A00101
Processing A00102
Processing A00107
Processing A00128
Processing A00132
Processing A00137
Processing A00141
Processing A00155
Processing A00156
Processing A00208
Processing A00216
Processing A00217
Processing A00225
Processing A00231
Processing A00247
Processing A00253
Processing A00264
Processing A00267
Processing A00271
Processing A00301
Processing A00321
Processing A00375
Processing A00395
Processing A00397
Processing A00404
Processing A00405
Processing A00422
Processing A00432
Processing A00438
Processing A00439
Processing A00441
Processing A00456
Processing A00465
Processing A00473
Processing A00486
Processing A00493
Processing A00509
Processing A00519
Processing A00520
Processing A00542
Processing A00551
Processing A00567
Processing A00587
Processing A00589
Processing A00592
Processing

### img save (All) : None-Polar(Square)

In [ ]:
# def save_scalogram(coefficients, save_path, id, cmap, image_size, dpi, dbsacle):
#     fig_size_inch = (image_size[0] / dpi, image_size[1] / dpi)
#     fig, ax = plt.subplots(figsize=fig_size_inch, dpi=dpi)
    
#     # Apply dB scale for the scalogram
#     if dbsacle:
#         energy = np.abs(coefficients) ** 2
#         energy_db = 10 * np.log10(energy + 1e-10)
#         c = ax.imshow(energy_db, cmap=cmap, aspect='auto')

#     # None db sacle
#     else:
    
#         c = ax.imshow(np.abs(coefficients), cmap=cmap, aspect='auto')
    
#     # Remove axis for clean image
#     ax.axis('off')
    
#     # Adjust layout to avoid padding
#     plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
#     ax.margins(0, 0)
#     ax.xaxis.set_major_locator(plt.NullLocator())
#     ax.yaxis.set_major_locator(plt.NullLocator())
    
#     # Save the scalogram image
#     plt.savefig(f'{save_path}/{id}.png', format='png', bbox_inches='tight', pad_inches=0, transparent=True)
#     plt.close()


# for label, patient_ids in dic_all.items():
#     if label == "Atrial Fibrillation":
#         folder_name = "A"
#     elif label == "Normal Sinus Rhythm":
#         folder_name = "N"
#     elif label == "Other Rhythm":
#         folder_name = "O"
#     elif label == "Noisy":
#         folder_name = "~"

#     save_path_directory = os.path.join(base_save_path, folder_name)
#     if not os.path.exists(save_path_directory):
#         os.makedirs(save_path_directory)

#     for patient_id in patient_ids:
#         save_filename = f'{patient_id}'
#         save_path_img = os.path.join(save_path_directory, f'{save_filename}.png')

#         if os.path.exists(save_path_img):
#             print(f"Skipping {patient_id}, already exists.")
#             continue

#         print(f"Processing {patient_id}")
        
#         # Load and preprocess ECG data
#         ecg_signal, fs = ecg_read(patient_id)
        
#         if np.isnan(ecg_signal).any():
#             ecg_signal = np.nan_to_num(ecg_signal, nan=0.0)

#         if PT_denoised:
#             pt_tompkins = Pan_tompkins(ecg_signal, fs).fit()
#             ecg_signal = pt_tompkins

#         if Nomaliztion:
#             ecg_signal = normalize_ecg(ecg_signal)                 

#         if target_fs != 300:
#             ecg_signal = resample_ecg_signal(ecg_signal, fs, target_fs)
#             fs = target_fs 

#         scales = np.arange(1, max_scale + 1)
#         wavelet_coeffs, freqs = pywt.cwt(ecg_signal, scales, wavelet, sampling_period=1/fs)

#         if np.isnan(wavelet_coeffs).any():
#             wavelet_coeffs = np.nan_to_num(wavelet_coeffs, nan=0.0)

#         # Save the scalogram
#         save_scalogram(wavelet_coeffs, save_path_directory, save_filename, cmap=cmap, image_size=image_size, dpi=dpi, dbsacle=dbscale)

### image count

In [ ]:
import os

os.chdir(base_save_path)

# 폴더명 목록
folders = ["A", "N", "O", "~"]

# 각 폴더의 PNG 파일 개수를 세는 함수
def count_png_files(folder_name):
    count = 0
    # 해당 폴더가 존재하는지 확인
    if os.path.exists(folder_name):
        # 폴더 내의 모든 파일 목록을 가져옴
        for file in os.listdir(folder_name):
            # 파일이 PNG 확장자를 가지는지 확인
            if file.endswith(".png"):
                count += 1
    return count

# 각 폴더의 PNG 파일 개수를 출력
total=0
for folder in folders:
    png_count = count_png_files(folder)
    total= png_count+ total
    print(f"Folder '{folder}': {png_count} PNG files")

print(total)    

### 파라미터 저장

In [ ]:
from datetime import datetime
# 생성일자
current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 파라미터와 생성일자를 텍스트 파일로 저장
params = {
    "sampling_fs": target_fs,
    "PT_denoised": PT_denoised,
    "padding_length_s": padding_length_s,
    "Nomaliztion": Nomaliztion,
    "alpha": alpha,
    "max_scale": max_scale,
    "wavelet": wavelet,
    "image_size": image_size,
    "dpi": dpi,
    "vmin": vmin,
    "vmax": vmax,
    "cmap": cmap,
    "generated_date": current_date,
    "dbscale": dbscale,
}

# 저장할 파일 경로 설정
save_path = path  # 텍스트 파일을 저장할 경로를 지정하세요
txt_file_path = os.path.join(save_path, '스케일로그램_nonpolar 설명.txt')

# 파일에 쓰기
with open(txt_file_path, 'w') as f:
    f.write("[ECG Polar Scalogram]\n")
    f.write(f"\n생성일자:{params['generated_date']}\n")
    f.write("\n")
    f.write("#########################\n")
    f.write("ECG 전처리 관련 파라미터:\n")
    f.write(f"1. sampling_fs: {params['sampling_fs']}\n")
    f.write(f"2. PT_denoised: {params['PT_denoised']}\n")
    f.write(f"3. padding_length_s: {params['padding_length_s']}\n")
    f.write(f"4. Nomaliztion : {params['Nomaliztion']}\n")
    f.write(f"5. alpha: {params['alpha']}\n")
    f.write("#########################")
    f.write("\nScalogram 관련 파라미터:\n")
    f.write(f"6. max_scale: {params['max_scale']}\n")
    f.write(f"7. wavelet: {params['wavelet']}\n")
    f.write("#########################")
    f.write("\n이미지 저장관련 파라미터:\n")
    f.write(f"8.image_size: {params['image_size']}\n")
    f.write(f"9. dpi: {params['dpi']}\n")
    f.write(f"10. vmin: {params['vmin']}\n")
    f.write(f"11. vmax: {params['vmax']}\n")
    f.write(f"12. cmap: {params['cmap']}\n")
    f.write("#########################")

print(f"Parameters and date saved to {txt_file_path}")